In [1]:
import numpy as np
import sys
from astroquery.jplhorizons import Horizons
import astropy.constants as const 
import swiftestio as swio

In [27]:
#Values from JPL Horizons
AU2M = const.au.value
GMSunSI = const.GM_sun.value
Rsun = const.R_sun.value
GC = const.G.value
JD = 86400
year = 365.25 * JD
c = 299792458.0

MU2KG   = GMSunSI / GC #Conversion from mass unit (G * Msun) to kg
DU2M   = AU2M      #Conversion from distance unit (AU) to meters
TU2S   = year       #Conversion from time unit (Julian Day) to seconds
GU     = GC / (DU2M**3 / (MU2KG * TU2S**2))

GMSun = GMSunSI / (DU2M**3 / TU2S**2)

# Solar oblateness values: From Mecheri et al. (2004), using Corbard (b) 2002 values (Table II)
J2 = 2.198e-7 * (Rsun / DU2M)**2
J4 = -4.805e-9 * (Rsun / DU2M)**4

npl = 10

tstart = '2021-01-28'
tend = '2021-01-29'
tstep = '1d'
planetid = {
   'mercury'     : '1',
   'venus'       : '2',
   'earthmoon'   : '3',
   'mars'        : '4',
   'jupiter'     : '5',
   'saturn'      : '6',
   'uranus'      : '7',
   'neptune'     : '8',
   'plutocharon' : '9'
}

#Planet Msun/M ratio
MSun_over_Mpl = {
   'mercury'     : 6023600.0,
   'venus'       : 408523.71,
   'earthmoon'   : 328900.56,
   'mars'        : 3098708.,
   'jupiter'     : 1047.3486,
   'saturn'      : 3497.898,
   'uranus'      : 22902.98,
   'neptune'     : 19412.24,
   'plutocharon' :  1.35e8
}

#Planet radii in meters
Rpl = {
   'mercury'     : 2439.4e3,
   'venus'       : 6051.8e3,
   'earthmoon'   : 6371.0084e3, # Earth only for radius
   'mars'        : 3389.50e3,
   'jupiter'     : 69911e3,
   'saturn'      : 58232.0e3,
   'uranus'      : 25362.e3,
   'neptune'     : 24622.e3,
   'plutocharon' : 1188.3e3
}

pdata = {}
plvec = {}
Rhill = {}

for key,val in planetid.items():
   pdata[key] = Horizons(id=val, id_type='majorbody',location='@sun',
            epochs={'start': tstart, 'stop': tend,
            'step': tstep})
   plvec[key] = np.array([pdata[key].vectors()['x'][0],
         pdata[key].vectors()['y'][0], 
         pdata[key].vectors()['z'][0], 
         pdata[key].vectors()['vx'][0], 
         pdata[key].vectors()['vy'][0], 
         pdata[key].vectors()['vz'][0] 
         ])
   Rhill[key] = pdata[key].elements()['a'][0] * (3 * MSun_over_Mpl[key])**(-1.0 / 3.0)
ver = np.sqrt(plvec['earthmoon'][3]**2 + plvec['earthmoon'][4]**2 + plvec['earthmoon'][5]**2)
ver

0.01746575523103058

In [28]:

for plid in plvec:
    plvec[plid][3:] *= year / JD
ver = np.sqrt(plvec['earthmoon'][3]**2 + plvec['earthmoon'][4]**2 + plvec['earthmoon'][5]**2)
ver

6.37936709813392